In [1]:
import glob
import os
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
data_dir = 'data_meg'
subj = "R2210"
dtype = "raw"
raw = mne.io.read_raw_fif('data_meg/R2210_raw_ica.fif', preload=True)
save_dir = 'data_meg'

Opening raw data file data_meg/R2210_raw_ica.fif...


/var/folders/4s/v49tr3_d3vn5jcm7fg7z3cv00000gp/T/ipykernel_59155/4196179896.py:11: RuntimeWarning: This filename (data_meg/R2210_raw_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif('data_meg/R2210_raw_ica.fif', preload=True)


    Range : 0 ... 2779999 =      0.000 ...  2779.999 secs
Ready.
Opening raw data file /Users/labadmin/meg_decoding/data_meg/R2210_raw_ica-1.fif...
    Range : 2780000 ... 2954999 =   2780.000 ...  2954.999 secs
Ready.
Reading 0 ... 2954999  =      0.000 ...  2954.999 secs...


In [2]:
raw.filter(1, 40, method='iir')

Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 1 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 40.00 Hz: -6.02, -6.02 dB



<Raw | R2210_raw_ica.fif, 193 x 2955000 (2955.0 s), ~4.25 GB, data loaded>

In [3]:
events = mne.find_events(raw, stim_channel='STI 014', output='onset', shortest_event=1)
event_id = {
    'start': 1,
    'move': 2,
    'reveal_red': 4,
    'reveal_white': 8,
    'done': 16,
}
epochs = mne.Epochs(raw, events, event_id, tmin=-0.2, tmax=1.0, baseline=(None, 0), preload=True)

print(epochs)

5438 events found on stim channel STI 014
Event IDs: [ 1  2  4  8 16]
Not setting metadata
5438 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 5438 events and 1201 original time points ...
0 bad epochs dropped
<Epochs | 5438 events (all good), -0.2 – 1 s (baseline -0.2 – 0 s), ~9.39 GB, data loaded,
 'start': 120
 'move': 4191
 'reveal_red': 617
 'reveal_white': 390
 'done': 120>


In [4]:
start_events = events[events[:, 2] == event_id['start']]
done_events = events[events[:, 2] == event_id['done']]
sfreq = raw.info['sfreq']  # Sampling frequency

# Initialize a list to store trial information
trial_info = []

done_idx = 0

for start_event in start_events:
    start_sample = start_event[0]
    
    # Find the corresponding 'done' event
    while done_idx < len(done_events) and done_events[done_idx, 0] <= start_sample:
        done_idx += 1
    if done_idx < len(done_events):
        done_sample = done_events[done_idx, 0]
        done_idx += 1

        # Calculate tmin and tmax for the epoch
        tmin = -0.2  # 0.2 s before 'start'
        tmax = (done_sample - start_sample) / sfreq + 1.0  # Duration from 'start' to 1 s after 'done'

        # Store trial information
        trial_info.append({
            'event_sample': start_sample,
            'tmin': tmin,
            'tmax': tmax
        })
max_tmax = max([info['tmax'] for info in trial_info])
print(f"Maximum epoch duration: {max_tmax:.2f} seconds")


Maximum epoch duration: 78.10 seconds


In [5]:
epochs_list = []

for idx, info in enumerate(trial_info):
    event_sample = info['event_sample']
    event_time = event_sample / sfreq
    tmin = -0.2  # As before
    tmax = info['tmax']
    
    # Create event array for this epoch
    event_array = np.array([[event_sample, 0, event_id['start']]])
    
    # Create epoch for this trial
    epoch = mne.Epochs(raw, event_array, event_id={'start': event_id['start']},
                       tmin=tmin, tmax=tmax, baseline=(None, 0), preload=True,
                       reject_by_annotation=False, verbose=False)
    
    # If the epoch was not dropped, add it to the list
    if len(epoch):
        epochs_list.append(epoch)
    else:
        print(f"Epoch at sample {event_sample} was still dropped.")

In [6]:
# Concatenate all epochs
if epochs_list:
    epochs = mne.concatenate_epochs(epochs_list)
else:
    raise ValueError("No epochs could be created.")

# Get the data array
data = epochs.get_data()

# Find the maximum number of time points across all epochs
n_times_list = [epoch.times.shape[0] for epoch in epochs_list]
max_n_times = max(n_times_list)

# Get the number of epochs and channels
n_epochs = len(epochs)
n_channels = len(epochs.ch_names)

In [9]:
# Initialize data array with NaNs
padded_data = np.full((n_epochs, n_channels, max_n_times), np.nan)

# Initialize mask array
valid_data_mask = np.zeros((n_epochs, n_channels, max_n_times), dtype=bool)

# Fill the data and mask arrays
for idx, epoch_data in enumerate(data):
    n_times = epoch_data.shape[1]
    padded_data[idx, :, :n_times] = epoch_data
    valid_data_mask[idx, :, :n_times] = True

masked_data = ma.masked_array(padded_data, mask=~valid_data_mask)

/var/folders/4s/v49tr3_d3vn5jcm7fg7z3cv00000gp/T/ipykernel_59155/4285002557.py:3: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs(epochs_list)


ValueError: operands could not be broadcast together with shapes (37155,) (27526,) 

In [25]:
epochs.plot(n_epochs=5, n_channels=30,events=True)


Not setting metadata
120 matching events found
No baseline correction applied
0 projection items activated


In [6]:
meg_data = epochs.get_data(picks=["mag", "grad"])